### Drivemızı ekliyoruz

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Dosyamızı da ekleyip alt klasötlerimizi ayırıyoruz.

In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras import layers, models
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob

In [ ]:
#import os
print(os.listdir("/content/drive/MyDrive/DerinOgrenme/Pneumonia/X-Ray/"))

data_dir = '/content/drive/MyDrive/DerinOgrenme/Pneumonia/X-Ray/'

img_height, img_width = 128, 128
data_path = "/content/drive/MyDrive/DerinOgrenme/Pneumonia/X-Ray"
viral_path = os.path.join(data_path, "Viral")
bacterial_path = os.path.join(data_path, "Bacterial")

['Viral', 'Bacterial']


### Görüntüyü okuyup boyutlandırıyoruz. Veri setini birleştirip, test ve eğitim olarak ayırıyoruz.

In [ ]:
def load_images(folder_path, label):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        if img is None:
            print(f"Error: Unable to read image {img_path}")
            continue
        img = cv2.resize(img, (img_height, img_width))
        images.append(img)
        labels.append(label)
    return np.array(images), np.array(labels)

viral_images, viral_labels = load_images(viral_path, 0)
bacterial_images, bacterial_labels = load_images(bacterial_path, 1)


x = np.concatenate((viral_images, bacterial_images), axis=0)
y = np.concatenate((viral_labels, bacterial_labels), axis=0)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state =42)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)

#----------------------------%20 CNN----------------------------


In [ ]:
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

x_val = np.array(x_val)
y_val = np.array(y_val)

In [ ]:
import tensorflow as tf

train_yCl = tf.keras.utils.to_categorical(y_train, num_classes=2)
test_yCl = tf.keras.utils.to_categorical(y_test, num_classes=2)
valid_yCl = tf.keras.utils.to_categorical(y_val, num_classes=2)

In [ ]:
pip install keras-tuner


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 1.0 MB/s eta 0:00:00


In [ ]:
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

def build_model(hp):
    model = Sequential()

    # Convolutional Layer 1
    model.add(Conv2D(filters=hp.Int('conv1_filters', min_value=32, max_value=128, step=16),
                     kernel_size=(3, 3),
                     padding='same',
                     input_shape=(img_height, img_width, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Convolutional Layer 2
    model.add(Conv2D(filters=hp.Int('conv2_filters', min_value=64, max_value=256, step=16),
                     kernel_size=(3, 3),
                     padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Flatten Layer
    model.add(Flatten())

    # Dense Layer 1
    model.add(Dense(units=hp.Int('dense1_units', min_value=256, max_value=1024, step=64)))
    model.add(Activation('relu'))
    model.add(Dropout(rate=hp.Float('dropout1', min_value=0.2, max_value=0.5, step=0.1)))

    # Dense Layer 2
    model.add(Dense(units=hp.Int('dense2_units', min_value=128, max_value=512, step=64)))
    model.add(Activation('relu'))
    model.add(Dropout(rate=hp.Float('dropout2', min_value=0.2, max_value=0.5, step=0.1)))

    # Output Layer
    model.add(Dense(2, activation='softmax'))

    # Modeli derleyin
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


<ipython-input-9-3c2453c56ba3>:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [ ]:
# Keras Tuner'ın RandomSearch sınıfını kullanarak tuner'ı oluşturun
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Denenecek maksimum model sayısı
    directory='my_dir',  # Kaydedilen modellerin ve sonuçların saklandığı dizin
    project_name='pneumonia_hyperparameter_tuning'  # Tuning projesinin adı
)

# Tuner'ı eğitim verisi ile uydurun
tuner.search(x_train, train_yCl, epochs=10, validation_data=(x_val, valid_yCl))

# En iyi hiperparametreleri ve modeli alın
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)

# Eğitim seti üzerinde en iyi modeli eğitin
best_model.fit(x_train, train_yCl, epochs=10, validation_data=(x_val, valid_yCl))

Trial 5 Complete [00h 03m 09s]
val_accuracy: 0.943511426448822

Best val_accuracy So Far: 0.9694656729698181
Total elapsed time: 00h 29m 59s
Epoch 1/10
369/369 [==============================] - 10s 22ms/step - loss: 6.5678 - accuracy: 0.7335 - val_loss: 0.4469 - val_accuracy: 0.8008
Epoch 2/10
369/369 [==============================] - 8s 22ms/step - loss: 0.3725 - accuracy: 0.8404 - val_loss: 0.2798 - val_accuracy: 0.8771
Epoch 3/10
369/369 [==============================] - 8s 21ms/step - loss: 0.2415 - accuracy: 0.9040 - val_loss: 0.2012 - val_accuracy: 0.9336
Epoch 4/10
369/369 [==============================] - 8s 22ms/step - loss: 0.1691 - accuracy: 0.9360 - val_loss: 0.1640 - val_accuracy: 0.9427
Epoch 5/10
369/369 [==============================] - 8s 21ms/step - loss: 0.1282 - accuracy: 0.9530 - val_loss: 0.1905 - val_accuracy: 0.9420
Epoch 6/10
369/369 [==============================] - 8s 22ms/step - loss: 0.0890 - accuracy: 0.9680 - val_loss: 0.0865 - val_accuracy: 0.9710


In [ ]:
score_valid = best_model.evaluate(x_val, valid_yCl)
print("Validation Accuracy: ", score_valid[1])

score_test = best_model.evaluate(x_test, test_yCl)
print("Test Accuracy: ", score_test[1])

score_train = best_model.evaluate(x_train, train_yCl)
print("Training Accuracy: ", score_train[1])

41/41 [==============================] - 0s 8ms/step - loss: 0.0796 - accuracy: 0.9733
Validation Accuracy:  0.9732824563980103
103/103 [==============================] - 1s 12ms/step - loss: 0.2164 - accuracy: 0.9585
Test Accuracy:  0.958473265171051
369/369 [==============================] - 2s 6ms/step - loss: 0.0353 - accuracy: 0.9878
Training Accuracy:  0.9877852201461792


In [ ]:
# Test verisi üzerinde tahminlerin yapılması
y_pred = best_model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(test_yCl, axis=1)

# Accuracy
accuracy = accuracy_score(y_true, y_pred_classes)
print(f'Test Accuracy: {accuracy:.4f}')

# Precision
precision = precision_score(y_true, y_pred_classes, average='weighted')
print(f'Precision: {precision:.4f}')

# Recall
recall = recall_score(y_true, y_pred_classes, average='weighted')
print(f'Recall: {recall:.4f}')

# F1 Score
f1 = f1_score(y_true, y_pred_classes, average='weighted')
print(f'F1 Score: {f1:.4f}')

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print('Confusion Matrix:')
print(conf_matrix)

# Classification Report
class_report = classification_report(y_true, y_pred_classes, labels=np.unique(y_true))
print('Classification Report:')
print(class_report)


103/103 [==============================] - 1s 6ms/step
Test Accuracy: 0.9585
Precision: 0.9589
Recall: 0.9585
F1 Score: 0.9586
Confusion Matrix:
[[1191   52]
 [  84 1948]]
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.96      0.95      1243
           1       0.97      0.96      0.97      2032

    accuracy                           0.96      3275
   macro avg       0.95      0.96      0.96      3275
weighted avg       0.96      0.96      0.96      3275



#----------------------------%20 YSA----------------------------


### Bir yapay sinir ağı oluşturuyoruz. 3 katman içeriyor. Giriş katmanı, Çıkış katmanı ve 2 gizli katman içermekte. İlk gizli katman 128 ikinci gizli katman 64 sinir ağı içermekte. Bazı metrikleri kullanarak modelimizi derliyoruz.


In [ ]:
# Model
model = models.Sequential()
model.add(layers.Flatten(input_shape=(img_height, img_width, 3)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Epoch 1/10
332/332 [==============================] - 4s 6ms/step - loss: 270.3531 - accuracy: 0.6274 - val_loss: 55.4873 - val_accuracy: 0.6556
Epoch 2/10
332/332 [==============================] - 2s 5ms/step - loss: 59.6184 - accuracy: 0.6532 - val_loss: 33.2392 - val_accuracy: 0.6768
Epoch 3/10
332/332 [==============================] - 2s 6ms/step - loss: 13.8717 - accuracy: 0.6687 - val_loss: 1.9813 - val_accuracy: 0.7354
Epoch 4/10
332/332 [==============================] - 2s 7ms/step - loss: 5.2549 - accuracy: 0.6453 - val_loss: 2.0600 - val_accuracy: 0.6429
Epoch 5/10
332/332 [==============================] - 2s 6ms/step - loss: 1.0735 - accuracy: 0.7106 - val_loss: 0.8464 - val_accuracy: 0.6735
Epoch 6/10
332/332 [==============================] - 2s 5ms/step - loss: 0.6679 - accuracy: 0.7218 - val_loss: 0.8428 - val_accuracy: 0.6421
Epoch 7/10
332/332 [==============================] - 2s 5ms/step - loss: 9.3919 - accuracy: 0.5963 - val_loss: 0.6804 - val_accuracy: 0.6175


### En son test verisi üzerinde modelimizi kullanıyoruz ve değerlendirmemizi alıyoruz.

In [ ]:
predictions = model.predict(x_test)
y_pred = (predictions > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

103/103 [==============================] - 0s 2ms/step
Accuracy: 0.6204580152671756
Precision: 0.6204580152671756
Recall: 1.0
F1 Score: 0.7657810439042774
Confusion Matrix:
[[   0 1243]
 [   0 2032]]
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1243
           1       0.62      1.00      0.77      2032

    accuracy                           0.62      3275
   macro avg       0.31      0.50      0.38      3275
weighted avg       0.38      0.62      0.48      3275



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#----------------------------%20 TL----------------------------

### DenseNet121

In [ ]:
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input
from tensorflow.keras.layers import GlobalMaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, Flatten

#Input layerımızı belirleyelim inputlarımızı densenet121 preprocess'ine uyduralım
input_layer = tf.keras.Input((img_width,img_height,3))
preprocessed_inputs = tf.keras.layers.Lambda( lambda x : preprocess_input(x))(input_layer)

#DenseNet121 için base model oluşturalım.
base_model = DenseNet121(
    weights = "imagenet",
    include_top = False, # Yalnızca Conv blocks kısmını include etsin.
    input_tensor = preprocessed_inputs # modele gelecek preprocessed inputlar
)

#Model outputuna Pooling ve Dense ekleyelim
k = GlobalAveragePooling2D()(base_model.output)
k = Dense(2)(k)

# Transfer modelimizin input ve output layerlarının set edilmesi
transfer_learning_model = tf.keras.Model(inputs = input_layer, outputs = k)

from tensorflow import keras
# Şimdi early stopping callbacklarimizi oluşturalım.
callback_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='model.h5',
        monitor = 'val_accuracy', save_best_only=True, verbose=3
    ),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=3)
]

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor = "val_loss",
    factor = 0.1,
    patience = 3,
    mode = "min",
    verbose = 1,
    min_lr = 1e-8
)

callback_list.append(reduce_lr)

#Modelimizi compile edelim
transfer_learning_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])

#Modelimizi eğitiyoruz.
transfer_learning_model.fit(
            x_train,
            train_yCl,
            epochs = 10,
            batch_size = 64,
            validation_data = (x_val, valid_yCl),
            callbacks = callback_list
            )

Epoch 1/10
185/185 [==============================] - ETA: 0s - loss: 5.5575 - accuracy: 0.6410
Epoch 1: val_accuracy improved from -inf to 0.37328, saving model to model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


185/185 [==============================] - 128s 285ms/step - loss: 5.5575 - accuracy: 0.6410 - val_loss: 9.6105 - val_accuracy: 0.3733 - lr: 0.0010
Epoch 2/10
185/185 [==============================] - ETA: 0s - loss: 5.3956 - accuracy: 0.6375
Epoch 2: val_accuracy improved from 0.37328 to 0.63053, saving model to model.h5
185/185 [==============================] - 43s 234ms/step - loss: 5.3956 - accuracy: 0.6375 - val_loss: 5.6665 - val_accuracy: 0.6305 - lr: 0.0010
Epoch 3/10
185/185 [==============================] - ETA: 0s - loss: 6.7610 - accuracy: 0.5386
Epoch 3: val_accuracy did not improve from 0.63053
185/185 [==============================] - 42s 228ms/step - loss: 6.7610 - accuracy: 0.5386 - val_loss: 5.9864 - val_accuracy: 0.6000 - lr: 0.0010
Epoch 4/10
185/185 [==============================] - ETA: 0s - loss: 5.8682 - accuracy: 0.5962
Epoch 4: val_accuracy did not improve from 0.63053
185/185 [==============================] - 42s 229ms/step - loss: 5.8682 - accuracy: 0.

### Test verisi üzerinde DenseNet121 modelimizi değerlendirelim

In [ ]:
#Modelimizin Validation, Test ve Train Acurracy Skorlarını hesaplayalım.
score_valid = transfer_learning_model.evaluate(x_val, valid_yCl)
print("Validation Accuracy: ", score_valid[1])

score_test = transfer_learning_model.evaluate(x_test, test_yCl)
print("Validation Accuracy: ", score_test[1])

score_train = transfer_learning_model.evaluate(x_train, train_yCl)
print("Validation Accuracy: ", score_train[1])

# Test verisi üzerinde tahminlerin yapılması
y_pred = transfer_learning_model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(test_yCl, axis=1)

# Accuracy
accuracy = accuracy_score(y_true, y_pred_classes)
print(f'Test Accuracy: {accuracy:.4f}')

# Precision
precision = precision_score(y_true, y_pred_classes, average='weighted')
print(f'Precision: {precision:.4f}')

# Recall
recall = recall_score(y_true, y_pred_classes, average='weighted')
print(f'Recall: {recall:.4f}')

# F1 Score
f1 = f1_score(y_true, y_pred_classes, average='weighted')
print(f'F1 Score: {f1:.4f}')

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print('Confusion Matrix:')
print(conf_matrix)

# Classification Report
class_report = classification_report(y_true, y_pred_classes, labels=np.unique(y_true))
print('Classification Report:')
print(class_report)

41/41 [==============================] - 4s 33ms/step - loss: 4.9998 - accuracy: 0.6702
Validation Accuracy:  0.6702290177345276
103/103 [==============================] - 5s 53ms/step - loss: 5.2265 - accuracy: 0.6547
Validation Accuracy:  0.6546564698219299
369/369 [==============================] - 12s 33ms/step - loss: 5.1408 - accuracy: 0.6593
Validation Accuracy:  0.6592586040496826
103/103 [==============================] - 5s 31ms/step
Test Accuracy: 0.6547
Precision: 0.6396
Recall: 0.6547
F1 Score: 0.6345
Confusion Matrix:
[[ 458  785]
 [ 346 1686]]
Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.37      0.45      1243
           1       0.68      0.83      0.75      2032

    accuracy                           0.65      3275
   macro avg       0.63      0.60      0.60      3275
weighted avg       0.64      0.65      0.63      3275



### EfficientNetB0

In [ ]:
from tensorflow.keras.applications import EfficientNetB0

# DenseNet121'de olduğu şekilde EfficientNetB0 modelimizi oluşturuyoruz.
base_model = EfficientNetB0(
    weights='imagenet',
    include_top=False,
    input_shape=(img_width, img_height, 3)
)

k = GlobalMaxPooling2D()(base_model.output)
k = Dense(2)(k)

transfer_learning_model_efficientnet = tf.keras.Model(inputs=base_model.input, outputs=k)

transfer_learning_model_efficientnet.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

callback_list_efficientnet = [
    keras.callbacks.ModelCheckpoint(
        filepath='model_efficientnet.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=3
    ),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=3)
]

transfer_learning_model_efficientnet.fit(
    x_train,
    train_yCl,
    epochs=10,
    batch_size=64,
    validation_data=(x_val, valid_yCl),
    callbacks=callback_list_efficientnet
)

Epoch 1/10
185/185 [==============================] - ETA: 0s - loss: 1.0301 - accuracy: 0.6419
Epoch 1: val_accuracy improved from -inf to 0.58397, saving model to model_efficientnet.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


185/185 [==============================] - 76s 202ms/step - loss: 1.0301 - accuracy: 0.6419 - val_loss: 0.6807 - val_accuracy: 0.5840
Epoch 2/10
185/185 [==============================] - ETA: 0s - loss: 0.6060 - accuracy: 0.7001
Epoch 2: val_accuracy improved from 0.58397 to 0.71221, saving model to model_efficientnet.h5
185/185 [==============================] - 33s 180ms/step - loss: 0.6060 - accuracy: 0.7001 - val_loss: 0.5592 - val_accuracy: 0.7122
Epoch 3/10
185/185 [==============================] - ETA: 0s - loss: 0.5146 - accuracy: 0.7705
Epoch 3: val_accuracy improved from 0.71221 to 0.78626, saving model to model_efficientnet.h5
185/185 [==============================] - 33s 179ms/step - loss: 0.5146 - accuracy: 0.7705 - val_loss: 0.4756 - val_accuracy: 0.7863
Epoch 4/10
185/185 [==============================] - ETA: 0s - loss: 0.5041 - accuracy: 0.7590
Epoch 4: val_accuracy did not improve from 0.78626
185/185 [==============================] - 32s 174ms/step - loss: 0.504

### Test verisi üzerinde EfficientNetB0 modelimizi değerlendirelim

In [ ]:
score_valid = transfer_learning_model.evaluate(x_val, valid_yCl)
print("Validation Accuracy: ", score_valid[1])

score_test = transfer_learning_model.evaluate(x_test, test_yCl)
print("Validation Accuracy: ", score_test[1])

score_train = transfer_learning_model.evaluate(x_train, train_yCl)
print("Validation Accuracy: ", score_train[1])

# Test verisi üzerinde tahminlerin yapılması
y_pred = transfer_learning_model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(test_yCl, axis=1)

# Accuracy
accuracy = accuracy_score(y_true, y_pred_classes)
print(f'Test Accuracy: {accuracy:.4f}')

# Precision
precision = precision_score(y_true, y_pred_classes, average='weighted')
print(f'Precision: {precision:.4f}')

# Recall
recall = recall_score(y_true, y_pred_classes, average='weighted')
print(f'Recall: {recall:.4f}')

# F1 Score
f1 = f1_score(y_true, y_pred_classes, average='weighted')
print(f'F1 Score: {f1:.4f}')

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print('Confusion Matrix:')
print(conf_matrix)

# Classification Report
class_report = classification_report(y_true, y_pred_classes, labels=np.unique(y_true))
print('Classification Report:')
print(class_report)

41/41 [==============================] - 2s 38ms/step - loss: 5.4128 - accuracy: 0.6626
Validation Accuracy:  0.6625953912734985
103/103 [==============================] - 3s 33ms/step - loss: 5.3909 - accuracy: 0.6660
Validation Accuracy:  0.6659541726112366
369/369 [==============================] - 13s 34ms/step - loss: 5.2749 - accuracy: 0.6754
Validation Accuracy:  0.6753753423690796
103/103 [==============================] - 3s 33ms/step
Test Accuracy: 0.6660
Precision: 0.6529
Recall: 0.6660
F1 Score: 0.6432
Confusion Matrix:
[[ 453  790]
 [ 304 1728]]
Classification Report:
              precision    recall  f1-score   support

           0       0.60      0.36      0.45      1243
           1       0.69      0.85      0.76      2032

    accuracy                           0.67      3275
   macro avg       0.64      0.61      0.61      3275
weighted avg       0.65      0.67      0.64      3275



#%35 ayırma

In [ ]:
# Veri setini ayırma
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.35, random_state =42)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)

#----------------------------%35 CNN----------------------------


In [ ]:
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

x_val = np.array(x_val)
y_val = np.array(y_val)

In [ ]:
import tensorflow as tf

train_yCl = tf.keras.utils.to_categorical(y_train, num_classes=2)
test_yCl = tf.keras.utils.to_categorical(y_test, num_classes=2)
valid_yCl = tf.keras.utils.to_categorical(y_val, num_classes=2)

In [ ]:
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

def build_model(hp):
    model = Sequential()

    # Convolutional Layer 1
    model.add(Conv2D(filters=hp.Int('conv1_filters', min_value=32, max_value=128, step=16),
                     kernel_size=(3, 3),
                     padding='same',
                     input_shape=(img_height, img_width, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Convolutional Layer 2
    model.add(Conv2D(filters=hp.Int('conv2_filters', min_value=64, max_value=256, step=16),
                     kernel_size=(3, 3),
                     padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Flatten Layer
    model.add(Flatten())

    # Dense Layer 1
    model.add(Dense(units=hp.Int('dense1_units', min_value=256, max_value=1024, step=64)))
    model.add(Activation('relu'))
    model.add(Dropout(rate=hp.Float('dropout1', min_value=0.2, max_value=0.5, step=0.1)))

    # Dense Layer 2
    model.add(Dense(units=hp.Int('dense2_units', min_value=128, max_value=512, step=64)))
    model.add(Activation('relu'))
    model.add(Dropout(rate=hp.Float('dropout2', min_value=0.2, max_value=0.5, step=0.1)))

    # Output Layer
    model.add(Dense(2, activation='softmax'))

    # Modeli derleyin
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


<ipython-input-9-3c2453c56ba3>:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [ ]:
# Keras Tuner'ın RandomSearch sınıfını kullanarak tuner'ı oluşturun
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Denenecek maksimum model sayısı
    directory='my_dir',  # Kaydedilen modellerin ve sonuçların saklandığı dizin
    project_name='pneumonia_hyperparameter_tuning'  # Tuning projesinin adı
)

# Tuner'ı eğitim verisi ile uydurun
tuner.search(x_train, train_yCl, epochs=10, validation_data=(x_val, valid_yCl))

# En iyi hiperparametreleri ve modeli alın
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)

# Eğitim seti üzerinde en iyi modeli eğitin
best_model.fit(x_train, train_yCl, epochs=10, validation_data=(x_val, valid_yCl))

Reloading Tuner from my_dir/pneumonia_hyperparameter_tuning/tuner0.json
Epoch 1/10
300/300 [==============================] - 13s 24ms/step - loss: 11.1968 - accuracy: 0.7366 - val_loss: 0.4471 - val_accuracy: 0.7878
Epoch 2/10
300/300 [==============================] - 6s 21ms/step - loss: 0.3675 - accuracy: 0.8426 - val_loss: 0.3388 - val_accuracy: 0.8563
Epoch 3/10
300/300 [==============================] - 7s 22ms/step - loss: 0.2388 - accuracy: 0.9037 - val_loss: 0.3452 - val_accuracy: 0.8854
Epoch 4/10
300/300 [==============================] - 6s 20ms/step - loss: 0.1846 - accuracy: 0.9276 - val_loss: 0.2819 - val_accuracy: 0.9052
Epoch 5/10
300/300 [==============================] - 6s 22ms/step - loss: 0.1497 - accuracy: 0.9451 - val_loss: 0.3936 - val_accuracy: 0.8761
Epoch 6/10
300/300 [==============================] - 6s 21ms/step - loss: 0.1299 - accuracy: 0.9510 - val_loss: 0.2880 - val_accuracy: 0.9474
Epoch 7/10
300/300 [==============================] - 6s 21ms/step -

In [ ]:
score_valid = best_model.evaluate(x_val, valid_yCl)
print("Validation Accuracy: ", score_valid[1])

score_test = best_model.evaluate(x_test, test_yCl)
print("Test Accuracy: ", score_test[1])

score_train = best_model.evaluate(x_train, train_yCl)
print("Training Accuracy: ", score_train[1])

34/34 [==============================] - 0s 8ms/step - loss: 0.3418 - accuracy: 0.9502
Validation Accuracy:  0.9502347707748413
180/180 [==============================] - 1s 7ms/step - loss: 0.3064 - accuracy: 0.9510
Test Accuracy:  0.9509684443473816
300/300 [==============================] - 2s 6ms/step - loss: 0.0183 - accuracy: 0.9928
Training Accuracy:  0.992796003818512


In [ ]:
# Test verisi üzerinde tahminlerin yapılması
y_pred = best_model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(test_yCl, axis=1)

# Accuracy
accuracy = accuracy_score(y_true, y_pred_classes)
print(f'Test Accuracy: {accuracy:.4f}')

# Precision
precision = precision_score(y_true, y_pred_classes, average='weighted')
print(f'Precision: {precision:.4f}')

# Recall
recall = recall_score(y_true, y_pred_classes, average='weighted')
print(f'Recall: {recall:.4f}')

# F1 Score
f1 = f1_score(y_true, y_pred_classes, average='weighted')
print(f'F1 Score: {f1:.4f}')

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print('Confusion Matrix:')
print(conf_matrix)

# Classification Report
class_report = classification_report(y_true, y_pred_classes, labels=np.unique(y_true))
print('Classification Report:')
print(class_report)


180/180 [==============================] - 1s 6ms/step
Test Accuracy: 0.9510
Precision: 0.9511
Recall: 0.9510
F1 Score: 0.9508
Confusion Matrix:
[[2018  185]
 [  96 3432]]
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.92      0.93      2203
           1       0.95      0.97      0.96      3528

    accuracy                           0.95      5731
   macro avg       0.95      0.94      0.95      5731
weighted avg       0.95      0.95      0.95      5731



#----------------------------%35 YSA----------------------------


In [ ]:
# Model
model = models.Sequential()
model.add(layers.Flatten(input_shape=(img_height, img_width, 3)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Epoch 1/10
270/270 [==============================] - 6s 8ms/step - loss: 172.3777 - accuracy: 0.6255 - val_loss: 755.4120 - val_accuracy: 0.6054
Epoch 2/10
270/270 [==============================] - 1s 5ms/step - loss: 73.9758 - accuracy: 0.6435 - val_loss: 61.4942 - val_accuracy: 0.6253
Epoch 3/10
270/270 [==============================] - 2s 6ms/step - loss: 24.8265 - accuracy: 0.6802 - val_loss: 7.5354 - val_accuracy: 0.7672
Epoch 4/10
270/270 [==============================] - 2s 6ms/step - loss: 12.5693 - accuracy: 0.6933 - val_loss: 17.1789 - val_accuracy: 0.5303
Epoch 5/10
270/270 [==============================] - 2s 7ms/step - loss: 5.8026 - accuracy: 0.7124 - val_loss: 4.0001 - val_accuracy: 0.7015
Epoch 6/10
270/270 [==============================] - 1s 5ms/step - loss: 1.8509 - accuracy: 0.7346 - val_loss: 1.8562 - val_accuracy: 0.6754
Epoch 7/10
270/270 [==============================] - 1s 5ms/step - loss: 12.7299 - accuracy: 0.6669 - val_loss: 6.1685 - val_accuracy: 0.7

In [ ]:
predictions = model.predict(x_test)
y_pred = (predictions > 0.5).astype(int)


accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)


print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

180/180 [==============================] - 0s 2ms/step
Accuracy: 0.5328912929680684
Precision: 0.9167482859941234
Recall: 0.2653061224489796
F1 Score: 0.41151901516816886
Confusion Matrix:
[[2118   85]
 [2592  936]]
Classification Report:
              precision    recall  f1-score   support

           0       0.45      0.96      0.61      2203
           1       0.92      0.27      0.41      3528

    accuracy                           0.53      5731
   macro avg       0.68      0.61      0.51      5731
weighted avg       0.74      0.53      0.49      5731



#----------------------------%35 TL----------------------------

### DenseNet121

In [ ]:
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input
from tensorflow.keras.layers import GlobalMaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, Flatten

input_layer = tf.keras.Input((img_width,img_height,3))
preprocessed_inputs = tf.keras.layers.Lambda( lambda x : preprocess_input(x))(input_layer)

base_model = DenseNet121(
    weights = "imagenet",
    include_top = False,
    input_tensor = preprocessed_inputs
)

k = GlobalAveragePooling2D()(base_model.output)
k = Dense(2)(k)

transfer_learning_model = tf.keras.Model(inputs = input_layer, outputs = k)

from tensorflow import keras

callback_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='model.h5',
        monitor = 'val_accuracy', save_best_only=True, verbose=3
    ),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=3)
]

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor = "val_loss",
    factor = 0.1,
    patience = 3,
    mode = "min",
    verbose = 1,
    min_lr = 1e-8
)

callback_list.append(reduce_lr)

transfer_learning_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])

#Modelimizi eğitiyoruz.
transfer_learning_model.fit(
            x_train,
            train_yCl,
            epochs = 10,
            batch_size = 64,
            validation_data = (x_val, valid_yCl),
            callbacks = callback_list
            )

Epoch 1/10
150/150 [==============================] - ETA: 0s - loss: 4.8711 - accuracy: 0.6746
Epoch 1: val_accuracy improved from -inf to 0.46197, saving model to model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


150/150 [==============================] - 129s 346ms/step - loss: 4.8711 - accuracy: 0.6746 - val_loss: 7.8859 - val_accuracy: 0.4620 - lr: 0.0010
Epoch 2/10
150/150 [==============================] - ETA: 0s - loss: 5.9923 - accuracy: 0.5958
Epoch 2: val_accuracy did not improve from 0.46197
150/150 [==============================] - 35s 231ms/step - loss: 5.9923 - accuracy: 0.5958 - val_loss: 9.1365 - val_accuracy: 0.3878 - lr: 0.0010
Epoch 3/10
150/150 [==============================] - ETA: 0s - loss: 5.3065 - accuracy: 0.6737
Epoch 3: val_accuracy improved from 0.46197 to 0.65164, saving model to model.h5
150/150 [==============================] - 36s 242ms/step - loss: 5.3065 - accuracy: 0.6737 - val_loss: 5.8291 - val_accuracy: 0.6516 - lr: 0.0010
Epoch 4/10
150/150 [==============================] - ETA: 0s - loss: 5.7312 - accuracy: 0.6270
Epoch 4: val_accuracy did not improve from 0.65164
150/150 [==============================] - 35s 233ms/step - loss: 5.7312 - accuracy: 0.

### Test verisi üzerinde DenseNet121 modelimizi değerlendirelim

In [ ]:
#Modelimizin Validation, Test ve Train Acurracy Skorlarını hesaplayalım.
score_valid = transfer_learning_model.evaluate(x_val, valid_yCl)
print("Validation Accuracy: ", score_valid[1])

score_test = transfer_learning_model.evaluate(x_test, test_yCl)
print("Validation Accuracy: ", score_test[1])

score_train = transfer_learning_model.evaluate(x_train, train_yCl)
print("Validation Accuracy: ", score_train[1])

# Test verisi üzerinde tahminlerin yapılması
y_pred = transfer_learning_model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(test_yCl, axis=1)

# Accuracy
accuracy = accuracy_score(y_true, y_pred_classes)
print(f'Test Accuracy: {accuracy:.4f}')

# Precision
precision = precision_score(y_true, y_pred_classes, average='weighted')
print(f'Precision: {precision:.4f}')

# Recall
recall = recall_score(y_true, y_pred_classes, average='weighted')
print(f'Recall: {recall:.4f}')

# F1 Score
f1 = f1_score(y_true, y_pred_classes, average='weighted')
print(f'F1 Score: {f1:.4f}')

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print('Confusion Matrix:')
print(conf_matrix)

# Classification Report
class_report = classification_report(y_true, y_pred_classes, labels=np.unique(y_true))
print('Classification Report:')
print(class_report)

34/34 [==============================] - 6s 100ms/step - loss: 5.9476 - accuracy: 0.6122
Validation Accuracy:  0.6122065782546997
180/180 [==============================] - 8s 45ms/step - loss: 5.8956 - accuracy: 0.6156
Validation Accuracy:  0.6155993938446045
300/300 [==============================] - 13s 42ms/step - loss: 5.7710 - accuracy: 0.6237
Validation Accuracy:  0.6237210035324097
180/180 [==============================] - 9s 32ms/step
Test Accuracy: 0.6156
Precision: 0.3790
Recall: 0.6156
F1 Score: 0.4691
Confusion Matrix:
[[   0 2203]
 [   0 3528]]
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2203
           1       0.62      1.00      0.76      3528

    accuracy                           0.62      5731
   macro avg       0.31      0.50      0.38      5731
weighted avg       0.38      0.62      0.47      5731



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:13

### EfficientNetB0

In [ ]:
from tensorflow.keras.applications import EfficientNetB0

base_model = EfficientNetB0(
    weights='imagenet',
    include_top=False,
    input_shape=(img_width, img_height, 3)
)

k = GlobalMaxPooling2D()(base_model.output)
k = Dense(2)(k)

transfer_learning_model_efficientnet = tf.keras.Model(inputs=base_model.input, outputs=k)

transfer_learning_model_efficientnet.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

callback_list_efficientnet = [
    keras.callbacks.ModelCheckpoint(
        filepath='model_efficientnet.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=3
    ),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=3)
]

transfer_learning_model_efficientnet.fit(
    x_train,
    train_yCl,
    epochs=10,
    batch_size=64,
    validation_data=(x_val, valid_yCl),
    callbacks=callback_list_efficientnet
)

Epoch 1/10
150/150 [==============================] - ETA: 0s - loss: 0.7023 - accuracy: 0.6722
Epoch 1: val_accuracy improved from -inf to 0.66948, saving model to model_efficientnet.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


150/150 [==============================] - 72s 226ms/step - loss: 0.7023 - accuracy: 0.6722 - val_loss: 0.5977 - val_accuracy: 0.6695
Epoch 2/10
150/150 [==============================] - ETA: 0s - loss: 0.5415 - accuracy: 0.7507
Epoch 2: val_accuracy improved from 0.66948 to 0.74648, saving model to model_efficientnet.h5
150/150 [==============================] - 27s 181ms/step - loss: 0.5415 - accuracy: 0.7507 - val_loss: 0.5420 - val_accuracy: 0.7465
Epoch 3/10
150/150 [==============================] - ETA: 0s - loss: 0.5122 - accuracy: 0.7734
Epoch 3: val_accuracy did not improve from 0.74648
150/150 [==============================] - 26s 175ms/step - loss: 0.5122 - accuracy: 0.7734 - val_loss: 0.5746 - val_accuracy: 0.7042
Epoch 4/10
150/150 [==============================] - ETA: 0s - loss: 0.5126 - accuracy: 0.7752
Epoch 4: val_accuracy did not improve from 0.74648
150/150 [==============================] - 26s 176ms/step - loss: 0.5126 - accuracy: 0.7752 - val_loss: 0.5607 - v

### Test verisi üzerinde EfficientNetB0 modelimizi değerlendirelim

In [ ]:
score_valid = transfer_learning_model.evaluate(x_val, valid_yCl)
print("Validation Accuracy: ", score_valid[1])

score_test = transfer_learning_model.evaluate(x_test, test_yCl)
print("Validation Accuracy: ", score_test[1])

score_train = transfer_learning_model.evaluate(x_train, train_yCl)
print("Validation Accuracy: ", score_train[1])

# Test verisi üzerinde tahminlerin yapılması
y_pred = transfer_learning_model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(test_yCl, axis=1)

# Accuracy
accuracy = accuracy_score(y_true, y_pred_classes)
print(f'Test Accuracy: {accuracy:.4f}')

# Precision
precision = precision_score(y_true, y_pred_classes, average='weighted')
print(f'Precision: {precision:.4f}')

# Recall
recall = recall_score(y_true, y_pred_classes, average='weighted')
print(f'Recall: {recall:.4f}')

# F1 Score
f1 = f1_score(y_true, y_pred_classes, average='weighted')
print(f'F1 Score: {f1:.4f}')

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print('Confusion Matrix:')
print(conf_matrix)

# Classification Report
class_report = classification_report(y_true, y_pred_classes, labels=np.unique(y_true))
print('Classification Report:')
print(class_report)

34/34 [==============================] - 1s 40ms/step - loss: 5.9476 - accuracy: 0.6122
Validation Accuracy:  0.6122065782546997
180/180 [==============================] - 6s 34ms/step - loss: 5.8956 - accuracy: 0.6156
Validation Accuracy:  0.6155993938446045
300/300 [==============================] - 10s 34ms/step - loss: 5.7710 - accuracy: 0.6237
Validation Accuracy:  0.6237210035324097
180/180 [==============================] - 6s 31ms/step
Test Accuracy: 0.6156
Precision: 0.3790
Recall: 0.6156
F1 Score: 0.4691
Confusion Matrix:
[[   0 2203]
 [   0 3528]]
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2203
           1       0.62      1.00      0.76      3528

    accuracy                           0.62      5731
   macro avg       0.31      0.50      0.38      5731
weighted avg       0.38      0.62      0.47      5731



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:13